# **@61**
# **Version 1.0**

**The following code mounts Google drive for use with Google CoLab**


In [ ]:
'''
Google Drive
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install utilsd2lzh_pytorch.py

In [ ]:
import time
import torch
from torch import nn, optim
import sys
sys.path.append("..")
import utilsd2lzh_pytorch as d2l #zip file added in project folder rename utils to utilsd2lzh_pytorch then add in colab directory
import PIL
device = torch.device('cuda' if torch.cuda.is_available() else'cpu')
#select device

print(torch.cuda.is_available())
from torchvision import transforms 
from torchvision import datasets
import os
from torchvision import models


#select device

device = torch.device("cuda:0")

#Perform different preprocessing on the three datasets to strengthen the training data

data_transforms = {

    'train': transforms.Compose([

        transforms.RandomRotation(30),

        transforms.RandomResizedCrop(224),

        transforms.RandomHorizontalFlip(),

        transforms.ToTensor(),

        transforms.Normalize([0.485, 0.456, 0.406],

                             [0.229, 0.224, 0.225])

    ]),

    'valid': transforms.Compose([

        transforms.Resize(256),

        transforms.CenterCrop(224),

        transforms.ToTensor(),

        transforms.Normalize([0.485, 0.456, 0.406],

                             [0.229, 0.224, 0.225])

    ]),

    'test': transforms.Compose([

        transforms.Resize(256),

        transforms.CenterCrop(224),

        transforms.ToTensor(),

        transforms.Normalize([0.485, 0.456, 0.406],

                             [0.229, 0.224, 0.225])

    ])

}

 

#data directory


data_dir = r"/content/drive/MyDrive/Datasets/51"

 

#Get three datasets

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),

                 data_transforms[x]) for x in ['train', 'valid','test']}

traindataset = image_datasets['train']

validdataset = image_datasets['valid']

testdataset = image_datasets['test']

 

batch_size = 16

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,

             shuffle=True, num_workers=2) for x in ['train', 'valid','test']}

print(dataloaders)

traindataloader = dataloaders['train']

validdataloader = dataloaders['valid']

testdataloader = dataloaders['test']

 

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid','test']}




In [ ]:
class Net(nn.Module):

    def __init__(self,model):

        super(Net,self).__init__()

        self.resnet = nn.Sequential(*list(model.children())[:-1])

        # Freeze parameters of convoluntional layers

        # for p in self.parameters():

        #     p.requires_grad = False

        self.fc = nn.Linear(in_features=2048,out_features=2)

        
    def forward(self,x):

        x = self.resnet(x)

        x = x.view(x.shape[0], -1)

        x = self.fc(x)

        return x

 
resnet152 = models.resnet152(pretrained=True)

net = Net(resnet152)

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),lr=0.0001,momentum=0.9)

In [ ]:
def valid_model(model, criterion):

    best_acc = 0.0

    # print('-' * 10)

 

    running_loss = 0.0

    running_corrects = 0

    model = model.to(device)

    for inputs, labels in validdataloader:

        inputs = inputs.to(device)

        labels = labels.to(device)

        model.eval()

        with torch.no_grad():

            outputs = model(inputs)

        loss = criterion(outputs, labels)

 

        _, preds = torch.max(outputs, 1)

        running_loss += loss.item()

        running_corrects += torch.sum(preds == labels)

    epoch_loss = running_loss / dataset_sizes['valid']

    # print(running_corrects.double())

    epoch_acc = running_corrects.double() / dataset_sizes['valid']

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(

            'valid', epoch_loss, epoch_acc))

    # print('-' * 10)

    print()

In [ ]:
def test_model(model, criterion):

    best_acc = 0.0

    # print('-' * 10)

 

    running_loss = 0.0

    running_corrects = 0

    model = model.to(device)

    for inputs, labels in testdataloader:

        inputs = inputs.to(device)

        labels = labels.to(device)

        model.eval()

        with torch.no_grad():

            outputs = model(inputs)

        loss = criterion(outputs, labels)

 

        _, preds = torch.max(outputs, 1)

        running_loss += loss.item()

        running_corrects += torch.sum(preds == labels)

    epoch_loss = running_loss / dataset_sizes['test']

    # print(running_corrects.double())

    epoch_acc = running_corrects.double() / dataset_sizes['test']

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(

            'test', epoch_loss, epoch_acc))

    # print('-' * 10)

    #print()

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=5):

    since = time.time()

    best_acc = 0.0

    for epoch in range(num_epochs):

        # since = time.time()
        
        # print('-' * 10)

        print('Epoch {}'.format(epoch))

 

        running_loss = 0.0

        running_corrects = 0

        model = model.to(device)

        for inputs, labels in traindataloader:

            inputs = inputs.to(device)

            labels = labels.to(device)

            model.train()

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = criterion(outputs, labels)

            loss.backward()

            optimizer.step()

 

            _, preds = torch.max(outputs, 1)

            running_loss += loss.item()

            running_corrects += torch.sum(preds == labels)

        epoch_loss = running_loss / dataset_sizes['train']

        # print(dataset_sizes['train'])

        # print(running_corrects.double())

        epoch_acc = running_corrects.double() / dataset_sizes['train']

        best_acc = max(best_acc,epoch_acc)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format('train', epoch_loss, epoch_acc))

        valid_model(model, criterion)
 
        time_elapsed = time.time() - since

        # print('use time: {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        # print()

 


    time_elapsed = time.time() - since

    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val Acc: {:4f}'.format(best_acc))

 

    return model

In [ ]:
epochs = 200
# model = torch.load(r'E:\Ud project\model1.pkl')
model = train_model(net, criterion, optimizer, epochs)
# model = train_model(model, criterion, optimizer, epochs)
test_model(model,criterion)
torch.save(model, r'/content/drive/MyDrive/Datasets/61/200_6_23.pkl')
torch.save(model, r'/content/drive/MyDrive/Datasets/61/200_7_23.model')

Epoch 0
train Loss: 0.0389 Acc: 0.6709
valid Loss: 0.0420 Acc: 0.6480

Epoch 1
train Loss: 0.0372 Acc: 0.6923
valid Loss: 0.0428 Acc: 0.6592

Epoch 2
train Loss: 0.0369 Acc: 0.6902
valid Loss: 0.0404 Acc: 0.6369

Epoch 3
train Loss: 0.0364 Acc: 0.6944
valid Loss: 0.0430 Acc: 0.6425

Epoch 4
train Loss: 0.0357 Acc: 0.7094
valid Loss: 0.0408 Acc: 0.6760

Epoch 5
train Loss: 0.0338 Acc: 0.7415
valid Loss: 0.0381 Acc: 0.6927

Epoch 6
train Loss: 0.0349 Acc: 0.7265
valid Loss: 0.0391 Acc: 0.7039

Epoch 7
train Loss: 0.0330 Acc: 0.7799
valid Loss: 0.0382 Acc: 0.7039

Epoch 8
train Loss: 0.0318 Acc: 0.7735
valid Loss: 0.0374 Acc: 0.7039

Epoch 9
train Loss: 0.0308 Acc: 0.7927
valid Loss: 0.0358 Acc: 0.7095

Epoch 10
train Loss: 0.0319 Acc: 0.7863
valid Loss: 0.0353 Acc: 0.7039

Epoch 11
train Loss: 0.0304 Acc: 0.8098
valid Loss: 0.0361 Acc: 0.7263

Epoch 12
train Loss: 0.0308 Acc: 0.7821
valid Loss: 0.0346 Acc: 0.7486

Epoch 13
train Loss: 0.0288 Acc: 0.8098
valid Loss: 0.0340 Acc: 0.7486

Ep